In [ ]:
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings
warnings.filterwarnings('ignore')

# Load the results first to get the significant genes
results_df = pd.read_csv("/workdir/execution_outputs/top_N_differentially_expressed_genes.csv")
print(f"Loaded results for {len(results_df)} genes")

# Load the original data
survival = pd.read_csv("/workdir/survival_KIRC.csv")
mrna = pd.read_csv("/workdir/KIRC_mRNA_top_column_cleaned.csv")
mrna = mrna.rename(columns={'unnamed_0': 'gene'})
mrna = mrna.set_index('gene')

# Prepare data for plotting
mrna_transposed = mrna.T
mrna_transposed.index.name = 'sample_name'
mrna_transposed = mrna_transposed.reset_index()
survival_mrna = survival.merge(mrna_transposed, on='sample_name', how='inner')

# Get top 5 genes with most significant survival associations
# Add hr_distance column for proper sorting
results_df['hr_distance'] = np.abs(results_df['hazard_ratio'] - 1)
results_df = results_df.sort_values(['cox_padj', 'hr_distance'], ascending=[True, False])

# Save the properly sorted results
results_df.to_csv('/workdir/execution_outputs/top_N_differentially_expressed_genes.csv', index=False)

top_5_genes = results_df.head(5)['gene'].tolist()
print(f"\nTop 5 genes for survival curve visualization:")
for i, gene in enumerate(top_5_genes, 1):
    gene_info = results_df[results_df['gene'] == gene].iloc[0]
    print(f"{i}. {gene} (HR: {gene_info['hazard_ratio']:.3f}, Cox p-adj: {gene_info['cox_padj']:.2e})")

def create_survival_plot(gene_name, survival_data, title_suffix=""):
    """Create survival plot for a single gene"""
    
    # Get gene expression
    gene_expr = survival_data[gene_name].values
    median_expr = np.median(gene_expr)
    
    # Create high/low expression groups
    high_mask = gene_expr >= median_expr
    low_mask = gene_expr < median_expr
    
    # Fit Kaplan-Meier curves
    kmf_high = KaplanMeierFitter()
    kmf_low = KaplanMeierFitter()
    
    kmf_high.fit(
        survival_data.loc[high_mask, 'survival_times'],
        survival_data.loc[high_mask, 'event_observed'],
        label='High Expression'
    )
    
    kmf_low.fit(
        survival_data.loc[low_mask, 'survival_times'],
        survival_data.loc[low_mask, 'event_observed'],
        label='Low Expression'
    )
    
    # Log-rank test
    logrank_results = logrank_test(
        survival_data.loc[high_mask, 'survival_times'],
        survival_data.loc[low_mask, 'survival_times'],
        survival_data.loc[high_mask, 'event_observed'],
        survival_data.loc[low_mask, 'event_observed']
    )
    
    # Get survival curves and confidence intervals
    times_high = kmf_high.timeline
    survival_high = kmf_high.survival_function_['High Expression']
    ci_high_lower = kmf_high.confidence_interval_.iloc[:, 0]
    ci_high_upper = kmf_high.confidence_interval_.iloc[:, 1]
    
    times_low = kmf_low.timeline
    survival_low = kmf_low.survival_function_['Low Expression']
    ci_low_lower = kmf_low.confidence_interval_.iloc[:, 0]
    ci_low_upper = kmf_low.confidence_interval_.iloc[:, 1]
    
    # Get gene statistics
    gene_stats = results_df[results_df['gene'] == gene_name].iloc[0]
    
    # Create figure
    fig = go.Figure()
    
    # High expression group
    fig.add_trace(go.Scatter(
        x=times_high,
        y=survival_high,
        mode='lines',
        name=f'High Expression (n={high_mask.sum()})',
        line=dict(color='red', width=2),
        showlegend=True
    ))
    
    # High expression confidence interval
    fig.add_trace(go.Scatter(
        x=times_high.tolist() + times_high.tolist()[::-1],
        y=ci_high_upper.tolist() + ci_high_lower.tolist()[::-1],
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='High Expression CI',
        showlegend=False
    ))
    
    # Low expression group
    fig.add_trace(go.Scatter(
        x=times_low,
        y=survival_low,
        mode='lines',
        name=f'Low Expression (n={low_mask.sum()})',
        line=dict(color='blue', width=2),
        showlegend=True
    ))
    
    # Low expression confidence interval
    fig.add_trace(go.Scatter(
        x=times_low.tolist() + times_low.tolist()[::-1],
        y=ci_low_upper.tolist() + ci_low_lower.tolist()[::-1],
        fill='toself',
        fillcolor='rgba(0,0,255,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Low Expression CI',
        showlegend=False
    ))
    
    # Update layout
    fig.update_layout(
        title=f'{gene_name} Survival Analysis{title_suffix}<br>' +
              f'HR: {gene_stats["hazard_ratio"]:.3f} ' +
              f'(95% CI: {gene_stats["ci_lower"]:.3f}-{gene_stats["ci_upper"]:.3f})<br>' +
              f'Cox p-value: {gene_stats["cox_pvalue"]:.2e}, ' +
              f'Log-rank p-value: {logrank_results.p_value:.2e}',
        xaxis_title='Time (days)',
        yaxis_title='Survival Probability',
        template='plotly_white',
        width=800,
        height=600,
        font=dict(size=12),
        legend=dict(x=0.7, y=0.95)
    )
    
    fig.update_xaxis(showgrid=True)
    fig.update_yaxis(showgrid=True, range=[0, 1])
    
    return fig

# Create survival plots for top 5 genes
print(f"\nCreating survival plots for top 5 genes...")

for i, gene in enumerate(top_5_genes):
    print(f"Creating plot {i+1}/5: {gene}")
    
    fig = create_survival_plot(gene, survival_mrna, f" (Rank #{results_df[results_df['gene'] == gene]['rank'].iloc[0]})")
    
    # Save individual plot
    filename = f'/workdir/execution_outputs/survival_plot_{gene.replace("/", "_")}.html'
    fig.write_html(filename)
    print(f"  Saved: survival_plot_{gene.replace('/', '_')}.html")

# Create combined plot with subplots
print(f"\nCreating combined survival plot...")

fig_combined = make_subplots(
    rows=3, cols=2,
    subplot_titles=[f'{gene} (HR: {results_df[results_df["gene"] == gene]["hazard_ratio"].iloc[0]:.3f})' 
                   for gene in top_5_genes],
    vertical_spacing=0.08,
    horizontal_spacing=0.08
)

positions = [(1,1), (1,2), (2,1), (2,2), (3,1)]

for i, gene in enumerate(top_5_genes):
    row, col = positions[i]
    
    # Get gene expression and create groups
    gene_expr = survival_mrna[gene].values
    median_expr = np.median(gene_expr)
    high_mask = gene_expr >= median_expr
    low_mask = gene_expr < median_expr
    
    # Fit Kaplan-Meier curves
    kmf_high = KaplanMeierFitter()
    kmf_low = KaplanMeierFitter()
    
    kmf_high.fit(
        survival_mrna.loc[high_mask, 'survival_times'],
        survival_mrna.loc[high_mask, 'event_observed'],
        label='High'
    )
    
    kmf_low.fit(
        survival_mrna.loc[low_mask, 'survival_times'],
        survival_mrna.loc[low_mask, 'event_observed'],
        label='Low'
    )
    
    # Add traces to subplot
    fig_combined.add_trace(
        go.Scatter(
            x=kmf_high.timeline,
            y=kmf_high.survival_function_['High'],
            mode='lines',
            name=f'High (n={high_mask.sum()})' if i == 0 else None,
            line=dict(color='red', width=2),
            showlegend=(i == 0),
            legendgroup='high'
        ),
        row=row, col=col
    )
    
    fig_combined.add_trace(
        go.Scatter(
            x=kmf_low.timeline,
            y=kmf_low.survival_function_['Low'],
            mode='lines',
            name=f'Low (n={low_mask.sum()})' if i == 0 else None,
            line=dict(color='blue', width=2),
            showlegend=(i == 0),
            legendgroup='low'
        ),
        row=row, col=col
    )

fig_combined.update_layout(
    title='Top 5 Genes: Survival Analysis (High vs Low Expression)',
    template='plotly_white',
    height=900,
    width=1200,
    font=dict(size=10)
)

# Update all x and y axes
for i in range(1, 4):
    for j in range(1, 3):
        if i == 3 and j == 2:  # Skip the empty subplot
            continue
        fig_combined.update_xaxis(title_text='Time (days)', showgrid=True, row=i, col=j)
        fig_combined.update_yaxis(title_text='Survival Probability', showgrid=True, range=[0, 1], row=i, col=j)

# Save combined plot
fig_combined.write_html('/workdir/execution_outputs/combined_survival_plots_top5.html')
print(f"Saved: combined_survival_plots_top5.html")

# Create summary table of the top 5 genes
summary_data = []
for gene in top_5_genes:
    gene_info = results_df[results_df['gene'] == gene].iloc[0]
    summary_data.append({
        'Gene': gene,
        'Rank': int(gene_info['rank']),
        'Hazard_Ratio': round(gene_info['hazard_ratio'], 3),
        'CI_Lower': round(gene_info['ci_lower'], 3),
        'CI_Upper': round(gene_info['ci_upper'], 3),
        'Cox_P_Value': f"{gene_info['cox_pvalue']:.2e}",
        'Cox_P_Adjusted': f"{gene_info['cox_padj']:.2e}",
        'LogRank_P_Value': f"{gene_info['logrank_pvalue']:.2e}",
        'LogRank_P_Adjusted': f"{gene_info['logrank_padj']:.2e}",
        'Median_Survival_High': gene_info['median_survival_high'],
        'Median_Survival_Low': gene_info['median_survival_low']
    })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('/workdir/execution_outputs/top_5_genes_summary.csv', index=False)

print(f"\nSummary of Top 5 Genes:")
print(summary_df.to_string(index=False))
print(f"\nSaved: top_5_genes_summary.csv")

print(f"\nAll files created successfully!")
print(f"Main outputs:")
print(f"- top_N_differentially_expressed_genes.csv: All 100 genes analyzed")
print(f"- combined_survival_plots_top5.html: Combined survival plots")
print(f"- Individual survival plots for each of the top 5 genes")
print(f"- top_5_genes_summary.csv: Summary statistics for top 5 genes")
